In [ ]:
from espm.weights import generate_weights as gw
from espm.models import generate_EDXS_phases as edx
from espm.datasets import base as gen
from espm.conf import DATASETS_PATH
from espm.estimators.smooth_nmf import SmoothNMF
from espm.measures import find_min_config
from sklearn.decomposition import NMF
import matplotlib.pyplot as plt
import numpy as np
import hyperspy.api as hs

# Generate the distribution of phases

The generated phases are two spherical particles embedded in a matrix material. The seed was chosen so that both particles are fully inside the field of view and there is some small overlap between the particles.

In [ ]:
spheres = gw.generate_weights("sphere", [128,128],3,6549, radius =25.0)

## Plot the distribution of the phases

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(15,5))
cax = fig.add_axes([0.15, 0.1, 0.2, 0.03])
for i,weight in enumerate(spheres.T):
    im = ax[i].imshow(weight[:,:], cmap = 'hot', vmin = 0.0, vmax = 1.0)
    ax[i].set_title("Map of phase {}".format(i+1))
    ax[i].tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, right=False, left=False, labelleft=False)
fig.colorbar(im, cax=cax, orientation='horizontal')

# Generate the spectra of the phases

The matrix corresponds to a pseudo-bridgmanite while the two other phases correspond to pseudo-Ca-perovsite and pseudo-ferropericlase.

In [ ]:
elts_dicts = [
    {
        "Mg" : 0.245, "Fe" : 0.035, "Ca" : 0.031, "Si" : 0.219, "Al" : 0.024, "O" : 0.436, "Cu" : 0.05, "Hf" : 0.01
    },
    {
        "Mg" : 0.522, "Fe" : 0.104, "O" : 0.374, "Cu" : 0.05
    },
    {
        "Mg" : 0.020, "Fe" : 0.018, "Ca" : 0.188, "Si" : 0.173, "Al" : 0.010, "O" : 0.572, "Ti" : 0.004, "Cu" : 0.05, "Sm" : 0.007, "Lu" : 0.006, "Nd" : 0.006 
    }]

brstlg_pars = [
    {"b0" : 0.0003458, "b1" : 0.0006268},
    {"b0" : 0.0001629, "b1" : 0.0009812},
    {"b0" : 0.0007853, "b1" : 0.0003658}
]

model_params = {
        "e_offset" : 0.3,
        "e_size" : 1980,
        "e_scale" : 0.01,
        "width_slope" : 0.01,
        "width_intercept" : 0.065,
        "db_name" : "200keV_xrays.json",
        "E0" : 200,
        "params_dict" : {
            "Abs" : {
                "thickness" : 100.0e-7,
                "toa" : 35,
                "density" : 4.5,
                "atomic_fraction" : False
            },
            "Det" : "SDD_efficiency.txt"
        }
    }

phases = edx.generate_modular_phases (elts_dicts = elts_dicts, brstlg_pars = brstlg_pars, scales = [1,1,1], model_params = model_params, seed = 0)

## Plot the spectra of the phases

In [ ]:
x = np.linspace(model_params["e_offset"], model_params["e_offset"] + model_params["e_size"]*model_params["e_scale"], model_params["e_size"])
fig, ax = plt.subplots(1,3,figsize=(15,5))
for i in range(3):  
    ax[i].plot(x,phases[i],'k')
    ax[i].set_title("Phase {}".format(i+1))
    ax[i].set_ylabel("Intensity (a.u.)")
    ax[i].set_xlim([0,10])
    ax[i].set_xlabel("Energy (keV)")

# Generate the data

The datacubes are generated as tensorial product of the phases by the maps.

## Generate the low noise data

A N = 293 is chosen to create a dataset with a strong signal to noise ratio. N = 293 means that in average (modulated by the densities, see misc_params) the number of counts per spectrum in the datacube is 293.

In [ ]:
misc_params = {
    "data_folder" : "spheres_paper_N293",
    "shape_2d" : (128,128),
    "N" : 293,
    "densities" : [1.0,0.8,1.2],
    "model" : "EDXS",
    "seed" : 42
}
    
gen.generate_dataset(phases, spheres, model_params, misc_params, sample_number=5, base_seed = 42,
                      elements = ["O","Ca","Si","Fe","Mg","Al","Cu","Hf","Ti","Sm","Lu","Nd"])

## Generate the high noise data

Here N = 18.

In [ ]:
misc_params = {
    "data_folder" : "spheres_paper_N18",
    "shape_2d" : (128,128),
    "N" : 18,
    "densities" : [1.0,0.8,1.2],
    "model" : "EDXS",
    "seed" : 42
}
    
gen.generate_dataset(phases, spheres, model_params, misc_params, sample_number=5, base_seed = 42,
                      elements = ["O","Ca","Si","Fe","Mg","Al","Cu","Hf","Ti","Sm","Lu","Nd"])

# Analysis of the synthetic datacubes

## Analysis of the low noise data

### Load the generated data
We immediately copy the data so that we can perform the decomposition twice. Once with the espm implementation of NMF and once with the scikit-learn implementation of NMF

In [ ]:
espm_decomp_spim = hs.load(str(DATASETS_PATH) + "/spheres_paper_N293/sample_0.hspy")
espm_decomp_spim.change_dtype("float64")

scikit_decomp_spim = espm_decomp_spim.deepcopy()

### Generate the G matrix

The ```build_G``` function instensiate the G attribute. This attribute contains the G matrix used to perform the physics-based decomposition.

In [ ]:
espm_decomp_spim.build_G()

### Create the estimators objects to run the algorithms

#### espm implementation

Using the ```SmoothNMF``` object you can input all the parameters to perform your decomposition with the same framework as the scikit-learn one. For an explanation about the parameters see the documentation.

In [ ]:
espm_nmf = SmoothNMF(n_components=3,
                    G = espm_decomp_spim.G,
                    max_iter=10000,
                    tol=1e-8,
                    init = 'nndsvdar',
                    random_state = 42,
                    mu = 0.008,
                    epsilon_reg = 0.01,
                    lambda_L = 0.1,
                    shape_2d = (128,128),
                    normalize = True, hspy_comp = True)

Running the espm decomposition. /!\ This will take a while. /!\

In [ ]:
espm_decomp_spim.decomposition(algorithm = espm_nmf)

### scikit-learn implementation

In [ ]:
sk_nmf = NMF(n_components=3,init='nndsvdar',max_iter=10000,tol=1e-8, random_state = 42, beta_loss='kullback-leibler', solver='mu')

Running the scikit-learn decomposition. /!\ This will take a while. /!\

In [ ]:
scikit_decomp_spim.decomposition(algorithm = sk_nmf)

### Getting the results of the decompositions

We store the output of the decompositions as well as the ground truth into series of maps and spectra. We then find which map (or spectrum) correspond to which ground ground truth using `find_min_config`.

In [ ]:
espm_decomp_maps = espm_decomp_spim.get_decomposition_loadings().data.reshape((3,128*128))
espm_decomp_spectra = espm_decomp_spim.get_decomposition_factors().data

sk_decomp_maps = scikit_decomp_spim.get_decomposition_loadings().data.reshape((3,128*128))
sk_decomp_spectra = scikit_decomp_spim.get_decomposition_factors().data

true_maps = espm_decomp_spim.maps
true_spectra = espm_decomp_spim.phases.T

espm_config = find_min_config(true_maps, true_spectra, espm_decomp_maps, espm_decomp_spectra)[2]
sk_config = find_min_config(true_maps, true_spectra, sk_decomp_maps, sk_decomp_spectra)[2]

### Plotting the results

In [ ]:
fig, axs = plt.subplots(3,4,figsize=(15,9), width_ratios=[1,1,1,3])
cax = fig.add_axes([0.23, 0.08, 0.2, 0.01])
for i in range(3) :  
        im = axs[i,0].imshow(true_maps[i].reshape((128,128)),cmap='hot',vmin = 0, vmax = 1.0, aspect = 'auto')
        axs[i,0].tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, right=False, left=False, labelleft=False)
        axs[i,1].tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, right=False, left=False, labelleft=False)
        axs[i,2].tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, right=False, left=False, labelleft=False)
        axs[0,0].set_title("Ground truth")
        axs[i,0].set_ylabel("Phase {}".format(i+1))
        axs[i,2].imshow(espm_decomp_maps[espm_config[i]].reshape((128,128)),cmap='hot',vmin = 0, vmax = 1.0, aspect = 'auto')
        axs[0,1].set_title("Scikit decomposition")
        axs[i,1].imshow(sk_decomp_maps[sk_config[i]].reshape((128,128)),cmap='hot',vmin = 0, vmax = 1.0, aspect = 'auto')
        axs[0,2].set_title("espm decomposition")
        axs[i,3].plot(x,true_spectra[i] + 10,'k', label = 'Ground truth')
        axs[i,3].plot(x,sk_decomp_spectra[sk_config[i]] + 10,'r--', label = 'Scikit decomposition')
        axs[i,3].plot(x,true_spectra[i],'k')
        axs[i,3].plot(x,espm_decomp_spectra[espm_config[i]],'b--', label = 'espm decomposition')
        axs[i,3].set_xlim([0,10])
        axs[i,3].tick_params(axis = 'y', which = 'both', left = False, labelleft = False)
        axs[i,3].set_ylabel("Intensity (a.u.)")
        axs[0,3].legend()
        axs[2,3].set_xlabel("Energy (keV)")
fig.colorbar(im, cax=cax, orientation='horizontal')

## Analysis of the high noise data

In this part we follow the same steps but with adapted decomposition parameters. With that high level of noise, the results of the decompositions obviously worsen but they show that the physics modelling helps the analysis.

Note that depending on the initial random state, the results vary a lot.

### Load the generated data

In [ ]:
espm_decomp_spim_hn = hs.load(str(DATASETS_PATH) + "/spheres_paper_N18/sample_0.hspy")
espm_decomp_spim_hn.change_dtype("float64")

scikit_decomp_spim_hn = espm_decomp_spim_hn.deepcopy()

### Generate the G matrix

In [ ]:
espm_decomp_spim_hn.build_G()

### Create the estimators objects to run the algorithms

#### espm implementation

In [ ]:
espm_nmf_hn = SmoothNMF(n_components=3,
                    G = espm_decomp_spim_hn.G,
                    max_iter=10000,
                    tol=1e-8,
                    init = 'random',
                    random_state = 40,
                    mu = 0.00,
                    epsilon_reg = 1.0,
                    lambda_L = 0.0,
                    shape_2d = (128,128),
                    normalize = True, hspy_comp = True)

Running the espm decomposition. /!\ This will take a while. /!\

In [ ]:
espm_decomp_spim_hn.decomposition(algorithm = espm_nmf_hn)

### scikit-learn implementation

In [ ]:
sk_nmf_hn = NMF(n_components=3,init='random',max_iter=10000,tol=1e-8, random_state = 40, beta_loss='kullback-leibler', solver='mu')

Running the scikit-learn decomposition. /!\ This will take a while. /!\

In [ ]:
scikit_decomp_spim_hn.decomposition(algorithm = sk_nmf_hn)

### Getting the results of the decompositions

In [ ]:
espm_decomp_maps_hn = espm_decomp_spim_hn.get_decomposition_loadings().data.reshape((3,128*128))
espm_decomp_spectra_hn = espm_decomp_spim_hn.get_decomposition_factors().data

sk_decomp_maps_hn = scikit_decomp_spim_hn.get_decomposition_loadings().data.reshape((3,128*128))
sk_decomp_spectra_hn = scikit_decomp_spim_hn.get_decomposition_factors().data

true_maps_hn = espm_decomp_spim_hn.maps
true_spectra_hn = espm_decomp_spim_hn.phases.T

espm_config_hn = find_min_config(true_maps_hn, true_spectra_hn, espm_decomp_maps_hn, espm_decomp_spectra_hn, angles = False)[2]
sk_config_hn = find_min_config(true_maps_hn, true_spectra_hn, sk_decomp_maps_hn, sk_decomp_spectra_hn, angles = False)[2]

### Plotting the results

In [ ]:
fig, axs = plt.subplots(3,4,figsize=(15,9), width_ratios=[1,1,1,3])
cax = fig.add_axes([0.23, 0.08, 0.2, 0.01])
for i in range(3) :  
        im = axs[i,0].imshow(true_maps_hn[i].reshape((128,128)),cmap='hot',vmin = 0, vmax = 1.0, aspect = 'auto')
        axs[i,0].tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, right=False, left=False, labelleft=False)
        axs[i,1].tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, right=False, left=False, labelleft=False)
        axs[i,2].tick_params(axis='both', which='both', bottom=False, top=False, labelbottom=False, right=False, left=False, labelleft=False)
        axs[0,0].set_title("Ground truth")
        axs[i,0].set_ylabel("Phase {}".format(i+1))
        axs[i,2].imshow(espm_decomp_maps_hn[espm_config_hn[i]].reshape((128,128)),cmap='hot',vmin = 0, vmax = 1.0, aspect = 'auto')
        axs[0,1].set_title("Scikit decomposition")
        axs[i,1].imshow(sk_decomp_maps_hn[sk_config_hn[i]].reshape((128,128)),cmap='hot',vmin = 0, vmax = 1.0, aspect = 'auto')
        axs[0,2].set_title("espm decomposition")
        axs[i,3].plot(x,true_spectra_hn[i] + 2,'k', label = 'Ground truth')
        axs[i,3].plot(x,sk_decomp_spectra_hn[sk_config_hn[i]] + 2,'r--', label = 'Scikit decomposition')
        axs[i,3].plot(x,true_spectra_hn[i],'k')
        axs[i,3].plot(x,espm_decomp_spectra_hn[espm_config_hn[i]],'b--', label = 'espm decomposition')
        axs[i,3].set_xlim([0,10])
        axs[i,3].tick_params(axis = 'y', which = 'both', left = False, labelleft = False)
        axs[i,3].set_ylabel("Intensity (a.u.)")
        axs[0,3].legend()
        axs[2,3].set_xlabel("Energy (keV)")
fig.colorbar(im, cax=cax, orientation='horizontal')